In [1]:
import pandas as pd
import glob
import numpy as np

df = pd.read_csv('corpus.csv')

In [2]:
import gensim

# NLP stuff
import string
from hazm import *
import codecs

In [3]:
normalizer = Normalizer()
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stopwords.dat','r','utf-8').readlines()]
lemmatizer = Lemmatizer()

def preprocess(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stopwords]
    return tokens

In [4]:
data_preprocessed = df['beyt'].apply(preprocess)
data_preprocessed = data_preprocessed.tolist()

In [5]:
print(*data_preprocessed[:20], sep='\n')
print('doc len: ', len(data_preprocessed))

['با', 'نصرت', 'و', 'فتح', 'و', 'ظفر', 'و', 'دولت', 'والا', 'نگریست#نگر', 'علم', 'شاه', 'جهان', 'بر', 'سر', 'بالا']
['لشکر', 'شده', 'آسوده', 'و', 'ترمذ', 'شده', 'ایمن', 'نصرت', 'شده', 'پیوسته', 'و', 'دولت', 'شده', 'والا']
['فتح', 'آمده', 'و', 'تهنیت', 'آورده', 'جهان', 'را', 'سلطان', 'جهانگیر', 'به', 'این', 'فتح', 'مهنا']
['بشکفته', 'به', 'دین', 'داشت#دار', 'او', 'جان', 'پیمبر', 'نازنده', 'به', 'فرزند', 'او', 'آدم', 'و', 'حوا']
['بهروزی', 'او', 'در', 'همه', 'گیتی', 'شده', 'معروف', 'پیروزی', 'او', 'در', 'همه', 'عالم', 'شده', 'پیدا']
['رزم', 'همه', 'با', 'نصرت', 'و', 'رسم', 'همه', 'نیکو', 'روز', 'همه', 'با', 'دولت', 'و', 'کار', 'همه', 'زیبا']
['ای', 'شاه', 'غلا', 'تو', 'داشت#دار', 'به', 'اقطاع', 'چین', 'و', 'ختن', 'و', 'کاشغر', 'و', 'خلخ', 'و', 'یغما']
['بر', 'بیعت', 'و', 'پیمان', 'تو', 'صد', 'نامه', 'رسیدست', 'از', 'مکه', 'و', 'غزنین', 'و', 'سمرقند', 'و', 'بخارا']
['از', 'موکب', 'تو', 'کوه', 'نمود#نما', 'همه', 'هامون', 'وز', 'لشکر', 'تو', 'شهر', 'نمود#نما', 'همه', 'صحرا']
['آنجاکه', 'تف'

In [6]:
import logging
logging.basicConfig(filename='gensim.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.INFO)

## Using pre-trained

In [7]:
from gensim.test.utils import datapath
import fasttext.util
from gensim.models.fasttext import load_facebook_model, save_facebook_model

In [8]:
fasttext.util.download_model('fa', if_exists='ignore')
model = load_facebook_model(datapath("/home/poems/final_project/cc.fa.300.bin"))

In [30]:
model.build_vocab(data_preprocessed, update=True)
model.train(sentences=data_preprocessed, total_examples = len(data_preprocessed), epochs=100, workers = 11)

In [31]:
model.save('./results/fasttext')

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

def identity(x):
    return x

words_cv = CountVectorizer(max_features=10000, min_df=0.00001, max_df=0.3, tokenizer=identity, preprocessor=identity)
word_count_vector = words_cv.fit_transform(data_preprocessed)
word_tfidf = TfidfTransformer(smooth_idf=True)
word_tfidf = word_tfidf.fit(word_count_vector)

In [33]:
import dill

with open('./results/cv_tfidf.pkl', 'wb') as fout:
    dill.dump((words_cv, word_tfidf) , fout)

In [34]:
from sklearn.preprocessing import normalize as vec_normalize

words = words_cv.get_feature_names()
words_rep = model[words]
words_rep = vec_normalize(words_rep, axis=1)

/home/poems/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [35]:
doc_rep = word_count_vector @ words_rep 
norm_factor = np.sum(word_count_vector,axis=1)
norm_factor[norm_factor==0.]=1
doc_rep /= norm_factor

In [36]:
with open('./results/fasttext_doc_word_rep.pkl', 'wb') as fout:
    dill.dump((doc_rep, words_rep) , fout)

In [37]:
def find_best_match(query):
    query = word_tokenize(query)
    query = [lemmatizer.lemmatize(normalizer.normalize(y)) for y in query]
    query = words_cv.transform([query])
    query = word_tfidf.transform(query)
    query = query @ words_rep
    scores = doc_rep @ query.T
    return np.argsort(scores[:,0])[::-1]

In [38]:
indexes = find_best_match('شهریار زیر آفتاب خورشید ماهی خورد')

In [39]:
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 0)

df['beyt'][indexes[:20]]

362288    همی آفتاب فلک فروتاب زتاج تو گیردچو مه زآفتاب                  
2674      ماه است وزیر و ملک مشرق خورشید خورشید فروزنده بر ماه منیرست    
290317    برافرازنده چرخ مدور برافروزنده خورشید انور                     
492147    آسمانی و آفتابت صاحبست آفتابی کاسمانی چون تو کرد               
549123    خداوند خورشید و گردنده ماه فرازنده تاج و تخت و کلاه            
435039    ز پیشانیش تابان تیر و ناهید زر خسارش فروزان ماه و خورشید       
282285    چو خورد آن ماه را در آب ماهی ز ماهی ماه را چون بازخواهی        
222814    شاه بتانی و عاشقانت سپاهند ماه زمینی و آسمانت کلاهست           
108169    زیر دست چاکران شاه ماه و آفتاب زیر دست آفتاب و ماه چرخ و روزگار
557828    بیامد تهمتن بگسترد بر بخواهش بر شاه خورشید فر                  
127389    در سایه شاه آسمان قدر مه طلعت آفتاب پرتو                       
562566    بپرسیدش از شهریار و سپاه ز گردنده خورشید و تابنده ماه          
496439    آفتاب آسمانت باد تاج و آسمان آفتابت باد گاه                    
465399    ماه را از رخ چون خورشیدت در 

In [40]:
def get_precision_at_k_score(ground_truth, preds, k):
    ground_truth = set(ground_truth)
    preds = set(preds[:k])
    intersection = ground_truth.intersection(preds)
    return len(intersection) / min(k, len(ground_truth))

In [41]:
import pickle

In [42]:
with open('./eval/queries1.txt', encoding='utf-8') as f:
    queries1 = f.read().splitlines()
ranks1 = pickle.load(open('./eval/eval_ranks1.pkl', 'rb'))

In [43]:
with open('./eval/queries2.txt', encoding='utf-8') as f:
    queries2 = f.read().splitlines()
ranks2 = pickle.load(open('./eval/eval_ranks2.pkl', 'rb'))

In [44]:
queries = queries1 + queries2
ranks = ranks1 + ranks2

In [45]:
for k in range(1, 102, 20):
    prec = 0.0
    for i, q in enumerate(queries):
        prec+= get_precision_at_k_score(ranks[i], find_best_match(q), k)
    print(f"precision_at_k_score for k={k} is {prec/len(queries)}")

precision_at_k_score for k=1 is 0.06
precision_at_k_score for k=21 is 0.0870168165168165
precision_at_k_score for k=41 is 0.1179357569104179
precision_at_k_score for k=61 is 0.13810687438528121
precision_at_k_score for k=81 is 0.16506132211017593
precision_at_k_score for k=101 is 0.1863821661948636


## Training from Scratch

In [46]:
from gensim.models import FastText

model = FastText(data_preprocessed, size=100, window=5, min_count=5, workers=11,sg=1)

In [47]:
words = words_cv.get_feature_names()
words_rep = model[words]
words_rep = vec_normalize(words_rep, axis=1)

/home/poems/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [48]:
doc_rep = word_count_vector @ words_rep 
norm_factor = np.sum(word_count_vector,axis=1)
norm_factor[norm_factor==0.]=1
doc_rep /= norm_factor

In [49]:
def find_best_match(query):
    query = word_tokenize(query)
    query = [lemmatizer.lemmatize(normalizer.normalize(y)) for y in query]
    query = words_cv.transform([query])
    query = word_tfidf.transform(query)
    query = query @ words_rep
    scores = doc_rep @ query.T
    return np.argsort(scores[:,0])[::-1]

In [50]:
for k in range(1, 102, 20):
    prec = 0.0
    for i, q in enumerate(queries):
        prec+= get_precision_at_k_score(ranks[i], find_best_match(q), k)
    print(f"precision_at_k_score for k={k} is {prec/len(queries)}")

precision_at_k_score for k=1 is 0.03
precision_at_k_score for k=21 is 0.06106360306360305
precision_at_k_score for k=41 is 0.09265255340832988
precision_at_k_score for k=61 is 0.09590750442351309
precision_at_k_score for k=81 is 0.10587866795079709
precision_at_k_score for k=101 is 0.12962096239333176
